<a href="https://colab.research.google.com/github/Abhijeetkhade11/KaggleCompetitions/blob/main/ML_Experiment5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

In [ ]:
train = pd.read_csv("train (1).csv")
test = pd.read_csv("test (1).csv")



In [ ]:
target = "Heart Disease"

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
train.drop("id", axis=1, inplace=True)
test_ids = test["id"]
test.drop("id", axis=1, inplace=True)

In [ ]:
train.head()

In [ ]:
train["Age_group"] = pd.cut(train["Age"], bins=[0,40,55,65,100], labels=[0,1,2,3])
test["Age_group"] = pd.cut(test["Age"], bins=[0,40,55,65,100], labels=[0,1,2,3])


In [ ]:
train["chol_bp_ratio"] = train["Cholesterol"] / (train["BP"] + 1)
test["chol_bp_ratio"] = test["Cholesterol"] / (test["BP"] + 1)


In [ ]:
train["stress_index"] = train["ST depression"] * train["Max HR"]
test["stress_index"] = test["ST depression"] * test["Max HR"]


In [ ]:
X = train.drop("Heart Disease", axis=1)
y = train["Heart Disease"]

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:
xgb_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=6,
    eval_metric="logloss",
    use_label_encoder=False,
    enable_categorical=True
)

lgb_model = LGBMClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=-1
)

cat_model = CatBoostClassifier(
    iterations=500,
    learning_rate=0.05,
    depth=6,
    verbose=0
)

In [ ]:
oof_xgb = np.zeros(len(X))
oof_lgb = np.zeros(len(X))
oof_cat = np.zeros(len(X))

test_pred_xgb = np.zeros(len(test))
test_pred_lgb = np.zeros(len(test))
test_pred_cat = np.zeros(len(test))


In [ ]:
y_numeric = y.map({'Absence': 0, 'Presence': 1})

# Identify categorical features for CatBoost
cat_features = list(X.select_dtypes(include=['category', 'object']).columns)

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):

    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y_numeric.iloc[train_idx], y_numeric.iloc[val_idx]

    # XGBoost
    xgb_model.fit(X_train, y_train)
    oof_xgb[val_idx] = xgb_model.predict_proba(X_val)[:,1]
    test_pred_xgb += xgb_model.predict_proba(test)[:,1] / 5

    # LightGBM
    lgb_model.fit(X_train, y_train)
    oof_lgb[val_idx] = lgb_model.predict_proba(X_val)[:,1]
    test_pred_lgb += lgb_model.predict_proba(test)[:,1] / 5

    # CatBoost
    cat_model.fit(X_train, y_train, cat_features=cat_features)
    oof_cat[val_idx] = cat_model.predict_proba(X_val)[:,1]
    test_pred_cat += cat_model.predict_proba(test)[:,1] / 5

In [ ]:
print("XGB LogLoss:", log_loss(y, oof_xgb))
print("LGB LogLoss:", log_loss(y, oof_lgb))
print("CAT LogLoss:", log_loss(y, oof_cat))


In [ ]:
oof_blend = (oof_xgb + oof_lgb + oof_cat) / 3
print("Blended LogLoss:", log_loss(y, oof_blend))


In [ ]:
test_blend = (test_pred_xgb + test_pred_lgb + test_pred_cat) / 3


In [ ]:
submission = pd.DataFrame({
    "id": test_ids,
    "target": test_blend
})

submission.to_csv("submission.csv", index=False)


In [ ]:
submission.head()
